In [ ]:
import sys
from itertools import product, permutations

import numpy as np
import pandas as pd
from scipy.linalg import logm, eigh
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.cluster import SpectralClustering
import plotly.express as px

from tnpy.operators import SpinOperators, FullHamiltonian
from tnpy.model import Model1D, TotalSz
from tnpy.model.utils import boundary_vectors, minors_if_no_penalty
from tnpy.exact_diagonalization import ExactDiagonalization

np.set_printoptions(suppress=True, threshold=sys.maxsize, linewidth=sys.maxsize)

## Heisenberg model with random field

In [ ]:
class RandomHeisenberg(Model1D):
    def __init__(
        self,
        n: int,
        h: float,
        jt: float = 1,
        jz: float = 1,
        penalty: float = 0,
        s_target: int = 0,
        offset: float = 0,
        trial_id: str = None,
        seed: int = None,
    ):
        super().__init__(n)
        self._h = h
        self._jt = jt
        self._jz = jz
        self._penalty = penalty
        self._s_target = s_target
        self._offset = offset
        self._trial_id = trial_id
        self._seed = seed
        rng = np.random.RandomState(self.seed)
        self._random_sequence = rng.uniform(-self.h, self.h, size=self.n)

    @property
    def h(self) -> float:
        return self._h

    @property
    def jt(self) -> float:
        return self._jt

    @property
    def jz(self) -> float:
        return self._jz

    @property
    def penalty(self) -> float:
        return self._penalty

    @property
    def s_target(self) -> int:
        return self._s_target

    @property
    def offset(self) -> float:
        return self._offset

    @offset.setter
    def offset(self, offset: float):
        self._offset = offset

    @property
    def trial_id(self) -> str:
        return self._trial_id

    @boundary_vectors(row=0, col=-1)
    @minors_if_no_penalty(row=3, col=3)
    def _elem(self, site: int) -> np.ndarray:
        Sp, Sm, Sz, I2, O2 = SpinOperators()

        alpha = (
            self.penalty * (0.25 + self.s_target**2 / self.n) - self.offset / self.n
        )
        beta = self._random_sequence[site] - 2.0 * self.penalty * self.s_target

        return np.array(
            [
                [
                    I2,
                    0.5 * self.jt * Sp,
                    0.5 * self.jt * Sm,
                    2.0 * self.penalty * Sz,
                    self.jz * Sz,
                    alpha * I2 + beta * Sz,
                ],
                [O2, O2, O2, O2, O2, Sm],
                [O2, O2, O2, O2, O2, Sp],
                [O2, O2, O2, I2, O2, Sz],
                [O2, O2, O2, O2, O2, Sz],
                [O2, O2, O2, O2, O2, I2],
            ],
            dtype=float,
        )

    @property
    def seed(self) -> int:
        return self._seed

    @seed.setter
    def seed(self, seed: int) -> None:
        self._seed = seed
        rng = np.random.RandomState(self.seed)
        self._random_sequence = rng.uniform(-self.h, self.h, size=self.n)

## Exact diagonalization

In [ ]:
system_size = 4
model = RandomHeisenberg(n=system_size, jt=1, jz=1, h=1.0, seed=1993)
ed = ExactDiagonalization(model.mpo)

# print(ed.matrix, "\n")
# print(ed.evals,  "\n")
# print(ed.evecs,  "\n")

## Resolvent

In [ ]:
inv_mat = np.linalg.inv(-1 * ed.matrix)

print(inv_mat, "\n")
print(np.reciprocal(np.diag(inv_mat)))

## Bipartite entanglement entropy

In [ ]:
entropy = np.array(
            [
                ed.entanglement_entropy(ed.n_sites // 2, level_idx)
                for level_idx in range(len(ed.matrix))
            ]
        )

print(entropy)

## Spin sector

In [ ]:
_total_sz = FullHamiltonian(TotalSz(model.n).mpo).matrix
spin_sector = np.diag(ed.evecs.T @ _total_sz @ ed.evecs)

print(spin_sector)

## The standard Rayleigh-Ritz procedure

In [ ]:
u = np.array([[1, 0]])
d = np.array([[0, 1]])

def kron_product(ls):
    if not ls:
        return 1

    return np.kron(ls[0], kron_product(ls[1:]))

# for elem in product([u, d], repeat=model.n):
#     print(kron_product(elem))


In [ ]:
spin_zero_sequence = "{}{}".format("u" * (model.n // 2), "d" * (model.n // 2))
spin_zero_permutations = sorted(set(list(permutations(spin_zero_sequence, model.n))))
spin_map = {
    "u": u,
    "d": d
}

zero_charge_basis = [kron_product(list(map(spin_map.get, elem))) for elem in spin_zero_permutations]
proj = np.hstack(tuple(map(np.transpose, zero_charge_basis)))

# print(zero_charge_basis)
# print(proj)

In [ ]:
idendity = np.eye(2 ** 4)

# for i in range(4):
#     print(i, idendity.shape)
#     idendity = idendity.reshape(2, 2 ** ((4 - i - 1) + 4))
#     q, idendity = np.linalg.qr(idendity)
#     print(q, idendity)

In [ ]:
# zero_charge_basis = [
#     [u, u, d, d],
#     [u, d, u, d],
#     [u, d, d, u],
#     [d, u, u, d],
#     [d, u, d, u],
#     [d, d, u, u]
# ]

# proj = np.hstack(tuple(map(np.transpose, tuple(map(kron_product, zero_charge_basis)))))

reduced_mat = proj.T @ ed.matrix @ proj
evals, evecs = np.linalg.eigh(reduced_mat)

# print(ed.matrix, "\n")
# print(reduced_mat, "\n")
# print(evals, "\n")
# print(proj @ evecs, "\n")

In [ ]:
print([np.linalg.norm((ed.matrix - eval * np.eye(2 ** model.n)) @ proj @ evecs[:, idx]) for idx, eval in enumerate(evals)], "\n")
print(ed.evals, "\n")
print(ed.evecs, "\n")

In [ ]:
plt.matshow(ed.matrix)
plt.colorbar()
plt.matshow(reduced_mat)
plt.colorbar()

evecs = proj @ evecs
plt.matshow(evecs[:, 0:1] @ evecs[:, 0:1].T)
plt.colorbar()

## Spectral clustering

In [ ]:
diag = np.diagonal(reduced_mat)
off_diag = reduced_mat - np.diagflat(diag)
affinity = np.nan_to_num(off_diag * np.reciprocal(np.abs(diag[:, None] - diag)))
affinity = affinity / np.linalg.norm(affinity)
affinity_evals, affinity_evecs = np.linalg.eigh(affinity)

print(reduced_mat)
print(affinity)

In [ ]:
degree_mat = np.diagflat(np.sum(affinity, axis=0))
laplacian = degree_mat - affinity
lap_evals, lap_evecs = np.linalg.eigh(laplacian)

print(degree_mat)
print(laplacian)
print(lap_evals)
print(lap_evecs)

# Kirchhoff’s matrix-tree theorem
print(1/len(lap_evals) * np.prod(lap_evals[1:]))

In [ ]:
plt.matshow(reduced_mat)
plt.colorbar()
plt.matshow(affinity)
plt.colorbar()
plt.matshow(laplacian)
plt.colorbar()

In [ ]:
evals, evecs = np.linalg.eigh(reduced_mat)
affinity_evals, affinity_evecs = np.linalg.eigh(affinity)

print(evals, "\n")
print(evecs, "\n")
print(affinity_evals, "\n")
print(affinity_evecs, "\n")

## Graph with NetworkX

In [ ]:
full_ham_graph = nx.from_numpy_array(ed.matrix)
nx.draw(full_ham_graph, with_labels=True)
print(nx.clustering(full_ham_graph))

In [ ]:
ham_graph = nx.from_numpy_array(reduced_mat)
nx.draw(ham_graph, with_labels=True)

In [ ]:
graph = nx.from_numpy_array(affinity)
nx.draw(graph, with_labels=True)

In [ ]:
nx.draw_spectral(graph, with_labels=True)

In [ ]:
clustering = SpectralClustering(
    n_clusters=2,
    affinity="precomputed",
    assign_labels="discretize"
).fit(affinity)

clustering.labels_

## Spectral gap of Laplacian matrix

In [ ]:
def laplacian_evals(mat):
    diag = np.diagonal(mat)
    off_diag = mat - np.diagflat(diag)
    affinity = np.nan_to_num(off_diag * np.reciprocal(np.abs(diag[:, None] - diag)))
    # affinity = affinity / np.linalg.norm(affinity)
    degree_mat = np.diagflat(np.sum(affinity, axis=0))
    laplacian = degree_mat - affinity
    lap_evals, lap_evecs = np.linalg.eigh(laplacian)
    return lap_evals, lap_evecs

round_off_error = 1e-12

hs = np.linspace(1e-3, 20, 100)
seeds = np.arange(1000, 1020)
df = pd.DataFrame(columns=["h", "seed", "n_zeros", "spectral_gap", "fiedler_value", "fiedler_flips", "n_spanning_tree"])
for idx, (h, seed) in enumerate(product(hs, seeds)):
    model = RandomHeisenberg(n=system_size, jt=1, jz=1, h=h, seed=seed)
    reduced_mat = proj.T @ FullHamiltonian(model.mpo).matrix @ proj
    _lap_evals, _lap_evecs = laplacian_evals(reduced_mat)
    asign = np.sign(_lap_evecs[:, 1])
    signchange = ((np.roll(asign, 1) - asign) != 0).astype(int)
    df.loc[idx] = [
        h,
        seed,
        len(_lap_evals[_lap_evals < round_off_error]),
        _lap_evals[_lap_evals > round_off_error][0],
        _lap_evals[_lap_evals > round_off_error][1],
        np.sum(signchange),
        1/len(_lap_evals) * np.prod(_lap_evals[_lap_evals > round_off_error])
    ]

In [ ]:
df = df.groupby(["h"], as_index=False).mean()

fig = px.line(df, x="h", y="n_zeros", markers=True)
fig.show()
fig = px.line(df, x="h", y="spectral_gap", markers=True)
fig.show()
fig = px.line(df, x="h", y="fiedler_value", markers=True)
fig.show()
fig = px.line(df, x="h", y="fiedler_flips", markers=True)
fig.show()
fig = px.line(df, x="h", y="n_spanning_tree", markers=True, log_y=True)
fig.show()

**Note**: the Laplacian gap corresponding to h = 0 seems to be the Golden angle ??

### Test gap ratio

In [ ]:
def gap_ratio(evals):
    gap = np.diff(evals)
    next_gap = np.roll(gap, -1)
    return np.minimum(gap / next_gap, next_gap / gap)

avg_gap_ratios = []
for h in hs:
    model = RandomHeisenberg(n=system_size, jt=1, jz=1, h=h, seed=2023)
    reduced_mat = proj.T @ FullHamiltonian(model.mpo).matrix @ proj
    evals, evecs = np.linalg.eigh(reduced_mat)
    avg_gap_ratios.append(np.mean(gap_ratio(evals)))


fig = px.line(x=hs, y=avg_gap_ratios, markers=True)
fig.show()

### Test numpy random state

In [ ]:
rng = np.random.RandomState(seed=2000)
print(rng.uniform(-1, 1, size=4))

rng = np.random.RandomState(seed=2000)
print(rng.uniform(-2, 2, size=4))

## Harmonic Rayleigh-Ritz

In [ ]:
zero_charge_basis = [
    [u, u, d, d],
    [u, d, u, d],
    [u, d, d, u],
    [d, u, u, d],
    [d, u, d, u],
    [d, d, u, u]
]

proj = np.hstack(tuple(map(np.transpose, tuple(map(kron_product, zero_charge_basis)))))

shift = 16
shifted_mat = ed.matrix - shift * np.eye(len(ed.matrix))
evals, evecs = eigh(a=proj.T @ shifted_mat @ proj, b=proj.T @ np.linalg.matrix_power(shifted_mat, 2) @ proj)

evals = shift + np.reciprocal(evals)
evecs = shifted_mat @ proj @ evecs
residual_vec = np.stack([(ed.matrix - eval * np.eye(len(ed.matrix))) @ evecs[:, idx] for idx, eval in enumerate(evals)], axis=1)

print(evals, "\n")
print(evecs, "\n")
print(residual_vec, "\n")
print(np.linalg.norm(residual_vec, axis=0), "\n")

## Test against transverse Ising model

In [ ]:
class TransverseIsing(Model1D):
    def __init__(self, n, j, h):
        r"""

        .. math::

            H = -j \left( \sum_{i=0}^{N-1} S_{i+1}^z S_i^z + h S_i^x \right)

        Args:
            n: System size.
            j: Overall prefactor of energy.
            h: The coupling strength of transversed field.
        """
        super().__init__(n)
        self._j = j
        self._h = h

    @property
    def j(self):
        return self._j

    @property
    def h(self):
        return self._h

    @boundary_vectors(row=0, col=-1)
    def _elem(self, site: int) -> np.ndarray:
        Sp, Sm, Sz, I2, O2 = SpinOperators()
        Sx = Sp + Sm
        return np.array(
            [[I2, -self.j * Sz, -self.j * self.h * Sx], [O2, O2, Sz], [O2, O2, I2]],
            dtype=float,
        )

In [ ]:
model = TransverseIsing(n=4, j=1, h=0.5)
ed = ExactDiagonalization(model.mpo)

print(ed.evals, "\n")
print(ed.evecs , "\n")

plt.matshow(ed.matrix)
plt.colorbar()

In [ ]:
graph = nx.from_numpy_array(ed.matrix)
nx.draw(graph, with_labels=True)

In [ ]:
diag = np.diagonal(ed.matrix)
affinity = np.reciprocal(ed.matrix)
affinity[affinity == np.inf] = 0
np.fill_diagonal(affinity, 0)

plt.matshow(affinity)
plt.colorbar()

In [ ]:
diag = np.diagonal(ed.matrix)
off_diag = ed.matrix - np.diagflat(diag)
affinity = np.abs(off_diag) * np.reciprocal(np.abs(diag[:, None] - diag))
# affinity[np.nonzero(off_diag)] *= model.h
# affinity[np.where(off_diag == 0)] = 0
# affinity_evals, affinity_evecs = np.linalg.eigh(affinity)

# affinity = np.abs(np.linalg.inv(ed.matrix))
# affinity[affinity == np.inf] = 0
# affinity[affinity == -np.inf] = 0
# affinity = np.nan_to_num(affinity)
# np.fill_diagonal(affinity, 0)
print(affinity)

degree_mat = np.diagflat(np.sum(affinity, axis=0))
laplacian = degree_mat - affinity
lap_evals, lap_evecs = np.linalg.eigh(laplacian)

plt.matshow(affinity)
plt.colorbar()
plt.matshow(laplacian)
plt.colorbar()

In [ ]:
graph = nx.from_numpy_array(ed.matrix)
nx.draw(graph, with_labels=True)

In [ ]:
graph = nx.from_numpy_array(np.nan_to_num(affinity))
nx.draw(graph, with_labels=True)

In [ ]:
print(ed.evals, "\n")
print(ed.evecs, "\n")
print(lap_evals, "\n")
print(lap_evecs, "\n")

In [ ]:
hs = np.linspace(-2, 2, 100)
round_off_error = 1e-12

laplacian_gap = []
ns_spanning_tree = []
for h in hs:
    model = TransverseIsing(n=4, j=1, h=h)
    ed = ExactDiagonalization(model.mpo)
    # affinity = np.abs(np.linalg.inv(-ed.matrix))
    # np.fill_diagonal(affinity, 0)

    diag = np.diagonal(ed.matrix)
    off_diag = ed.matrix - np.diagflat(diag)
    affinity = np.reciprocal(np.abs(diag[:, None] - diag))

    affinity[np.nonzero(off_diag)] *= model.h
    affinity[np.where(off_diag == 0)] = 0
    affinity[affinity == np.inf] = 0
    affinity[affinity == -np.inf] = 0
    # affinity = np.nan_to_num(affinity)

    degree_mat = np.diagflat(np.sum(affinity, axis=0))
    laplacian = degree_mat - affinity
    lap_evals, lap_evecs = np.linalg.eigh(np.nan_to_num(laplacian))
    laplacian_gap.append(lap_evals[lap_evals > round_off_error][0])
    n_spanning_tree = 1/len(lap_evals) * np.prod(lap_evals[lap_evals > round_off_error])
    ns_spanning_tree.append(n_spanning_tree)

fig = px.line(x=hs, y=laplacian_gap, markers=True)
fig.show()
fig = px.line(x=hs, y=ns_spanning_tree, markers=True, log_y=True)
fig.show()